<img src="../images/RC_Logo_EN_0128px.png">

# Multi-curve bootstrapping

In [ ]:
#qgrid.nbinstall(overwrite=True)


In [ ]:
import datetime
from dateutil.relativedelta import relativedelta
import pyvacon.analytics as analytics
import pyvacon.tools.converter as converter
import pyvacon.tools.enums as enums
import pyvacon.marketdata.plot as mkt_plot
import pyvacon.marketdata.bootstrapping as bootstr
import math
import pandas
import qgrid
import pyvacon
# the next lin is a jupyter internal command to show the matplotlib graphs within the notebook
%matplotlib inline
import matplotlib.pyplot as plt

# alternative library for plots (offline version)
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True) 

## Introduction

In this notebook we introduce the basic principles of multi-curve bootstrapping. 

Before the credit crisis a single-curve framework was used for pricing interest rate derivatives. Both the forecasting of future cashflows as well as the discounting of future cashflows was based on the same curve. This curve was considered to be risk-free and was bootstrapped using a mixture of instruments indexed to rates with different tenors. 

Since the crisis, however, a widening of the spreads in tenor basis swaps has been observed. As a result, two interest rate swaps indexed e.g. to the 3M EURIBOR and 6M EURIBOR can no longer be priced using the same curve. Furthermore, a distinction needs to be made between the curve for the forward rates and the discount curve. This has led to the introduction of the multi-curve framework with separate curves for each tenor. 

In the multi-curve setup the curves are constructed based on instruments homogeneous in the referenced index. A set of instruments needs to be selected for each curve, with one instrument per maturity. Each instrument requires a quote as well as the respective instrument defintion, which is called a specification in the context of pyvacon.

## Input instruments for interest rate curve boostrapping
The following instruments are currently available for ir curve bootstrapping

- Deposits
- IR Futures
- IR Swaps
- IR Basis-Swaps
- FX Swaps



### Setting up deposits / fixings

The fixing of the underlying reference rate is published daily and represents a certain average rate earned over a period corresponding to the tenor. The rate is calculated from quotes obtained from a panel of selected banks. The start date (spot date) of the period can deviate from the fixing date. This difference is referred to as the spot lag. 

The fixing is usually used as the starting point for the bootstrapping of forward curves. It can be specified as a deposit in pyvacon (analytics.DepositSpecification).

In [ ]:
# calculation date
refdate_d = datetime.datetime(2017,8,31,0,0)
# dates entering analytics objects must be analytics ptimes
refdate = converter.getLTime(refdate_d)

# start date of the accrual period with spot lag equal to 2 days
startdate = converter.getLTime(2, refdate)

# end date of the accrual period is 1 day after startdate
enddate = converter.getLTime(1, startdate)

# specification of the deposit
deposit = analytics.DepositSpecification('OVERNIGHT_DEPOSIT', 'dummy_issuer', enums.SecuritizationLevel.NONE,
                                        'EUR', refdate, startdate, enddate, 100, 0.01, 
                                         enums.DayCounter.ACT365_FIXED)
    

### Setting up an interest rate swap
A plain vanilla interest rate swap is a financial contract in which a stream of fixed payments is exchanged for floating payments linked to a reference index. The par rate (r) of a swap is the fixed rate under which the value of the two streams (legs) is equal:
$$ r \cdot \sum_{i=1}^n dcf_{i} \cdot P(0,t_{i} ) = \sum_{k=1}^m F_{k} \cdot dcf_{k} \cdot P(0,t_{k}) $$ 

where $t_{i}$, $i=1,..,n$ and $t_{k}$, $i=1,..,m$ are the payment structures of the fixed and floating legs, and $P(0,t_{i/k})$ are the corresponding discount factors, $dcf_{i/k}$ is the day count fraction for the period $[t_{(i/k-1)},t_{i/k}]$, and $F_{k}$ is the expected value of  underlying reference rate for the period $[t_{(k-1)},t_{k}]$.

The standard payment frequency of the fixed leg depends on the currency of the swap as well as the tenor of the underlying. In the EUR market swaps are usually quoted with annual fixed payments.

The payment frequency of the floating leg usually coincides with the tenor of the underlying reference index. In some currencies, however, the floating rate can be compounded and payed out at less frequent intervals (e.g. CAD). 

In the context of pyvacon an IRS can be defined using analytics.InterestRateSwapSpecification.

In [ ]:
# start dates of the accrual periods corresponding to the tenor of the underlying index (3 months). The spot lag is set to 0.
start_dates_d = [refdate_d + relativedelta(months=3*i) for i in range(4)]
start_dates = converter.createPTimeList(refdate, start_dates_d)

# reset dates are equal to start dates if spot lag is 0.
reset_dates = start_dates

# the end dates of the accral periods
end_dates = converter.createPTimeList(refdate,[x+relativedelta(months=3) for x in start_dates_d])

# the actual payment dates of the cashflows may differ from the end of the accrual period (e.g. OIS). 
# in the standard case these two sets of dates coincide
pay_dates = end_dates
notionals = [1.0 for i in range(len(start_dates))] 

# definition of the floating leg
floatleg = analytics.IrFloatLegSpecification(notionals, reset_dates, start_dates, 
                                            end_dates, pay_dates,'EUR', 'dummy_udl', 
                                            enums.DayCounter.ACT365_FIXED, 0.0)
# definition of the fixed leg
fixedleg = analytics.IrFixedLegSpecification(0.01, notionals, start_dates, end_dates, 
                                       pay_dates,'EUR', enums.DayCounter.ACT365_FIXED)
# definition of the IR swap
ir_swap = analytics.InterestRateSwapSpecification('3M_SWAP', 'dummy_issuer', enums.SecuritizationLevel.COLLATERALIZED, 
                                                  'EUR', pay_dates[-1], fixedleg, floatleg)

### Setting up an overnight indexed swap
An OIS is an interest rate swap where the floating payments are linked to a compounded overnight rate. The floating paymets are obtained as:

 $$ N\prod_{i=1}^n (1+dcf_{i}\cdot I_{i-1})-1$$
where $N$ is the notional, $dcf_{i}$ is the day count fraction for the one-day period $[t_{i-1},t_{i}]$ and $I_{i-1}$ is the fixing of the ON rate corresponding to time $t_{i-1}$.

The overnight rate represents the default risk over one night and can, therefore, be regarded as mostly risk free. This rate is used to discount cashflows from collateralized trades as this rate is most often paid as interest for the collateral.

In the current pyvacon setting the OIS will be defined as plain vanilla IRS without compounding.

### Setting up a tenor basis swap

Basis swaps can be quoted in a fixed-fixed (as a portfolio of 2 fixed vs floating IRS) or in a float-float (as a single swap) convention. The market standard usually depends on the currency. 

#### Fixed-Fixed Basis Swaps (EUR convention)

In certain currencies (e.g EUR, SEK, DKK and NOK) basis swaps are quoted as the difference between two IRS with identical fixed legs and floating legs indexed to different tenors. With this convention the spread is usually paid annually, independent of the tenors of the floating legs. The par spread $s^{xy}$ is defined via:

$$ s^{xy} \cdot \sum_{l} dcf_{i} \cdot P(0,t_{l})= \sum_{k} dcf_{k} \cdot F_{k}^{y} \cdot P(0,t_{k})-\sum_{i} dcf_{i} \cdot F_{i}^{x} \cdot P(0,t_{i} )$$

where $t_{i}$, $t_{k}$   are grids corresponding to the tenors $x,y$ with $x<y$ , $t_{l}$ is the fixed payment grid.

#### Float-Float Basis Swaps

In case of a singel swap notation, the frequency of the floating payments usually coincides with the tenors of the reference rate. The spread is paid with the shorter leg. The par spread $s^{xy}$ is defined via:

$$\sum_{i} dcf_{i} \cdot (F_{i}^{x}+s^{xy} ) \cdot P(0,t_{i})= \sum_{k}dcf_{k} \cdot F_{k}^{y} \cdot P(0,t_{k})$$
where $t_{i}$, $t_{k}$ are grids corresponding to the tenors $x,y$ with $x<y$ .

For some currencies (USD, CAD) the floating leg corresponding to the shorter tenor is compounded to align the payment of both legs. In this case the payment frequency of the spread coincides with the longer tenor.

In pyvacon a basis swap can be specified using analytics.InterestRateBasisSwapSpecification. Here two floating legs corresponding to the two reference indices need to be defined along with a fixed leg corresponding to the spread payments.


In [ ]:
# we consider a 3M vs 6M basis swap. A pay and receive floating legs need to be specified (spread = receive - pay).
# The definition of the pay leg (3M) will be taken from the ir swap above 
floatleg_3M = floatleg

# get the 6M floating leg (receive leg)
start_dates_6M = converter.createPTimeList(refdate, [refdate_d + relativedelta(months=6*i) for i in range(2)])
reset_dates_6M = start_dates_6M
end_dates_6M = converter.createPTimeList(refdate, [refdate_d + relativedelta(months=6*(i+1)) for i in range(2)])
notionals_6M = [1.0 for i in range(len(start_dates_6M))]
floatleg_6M = analytics.IrFloatLegSpecification(notionals_6M, reset_dates_6M, start_dates_6M, end_dates_6M, end_dates_6M,
                                                'EUR', 'dummy_udl', enums.DayCounter.ACT365_FIXED, 0.0)

# the 3M leg is the payleg, 6M - the receve leg.
basis_swap = analytics.InterestRateBasisSwapSpecification('3M6M_BASIS_SWAP', 'test', enums.SecuritizationLevel.COLLATERALIZED, 
                                                  'EUR', end_dates_6M[-1], floatleg_3M, floatleg_6M,  fixedleg)

## Bootstrapping EUR curves
In the EUR market most interest rate derivatives are indexed to the Euribor benchmark rate and the Euro OverNight Index Average (Eonia) in case of Overnight indexed swaps (https://www.emmi-benchmarks.eu/emmi/about-us.html). Derivatives indexed to the 1M, 3M, 6M and 12M tenors can be found. 

In this notebook we present the construction of the OIS dicounting curve and the two most commonly used tenor curves - the 3M and 6M tenors.


The bootstrapping of different tenors needs to be performed sequentially. At first the OIS curve needs to be constructed, since it is used as the discount curve in the bootstraping algorithm for all other tenors. The OIS curve is calibrated under the assumption that the forward and discount curves coincide. 

The order of construction of the other tenors depends on the defined input instruments. If the derivatives only depend on one tenor (outright quotes), there are no restrictions on the order. However, for most currencies, one or two tenors will be more liquid (depending on the maturity) and all other tenors will be represented as basis swaps with respect to the main tenor. In this case the correct bootstrapping order needs to be followed and the corresponding basis index needs to be provided.

### Sample curve specification

In [ ]:
# sample ois curve
instruments = analytics.vectorBaseSpecification(2)
instruments[0] = deposit
instruments[1] = ir_swap 
quotes = analytics.vectorDouble([0.0025, 0.005])
eonia = analytics.YieldCurveBootstrapper.compute(refdate, 'EONIA_DC', enums.DayCounter.ACT365_FIXED,  instruments, quotes)

In [ ]:
# sample 3M EURIBOR curve with ois discounting (the same instruments are used for simplification)
quotes = analytics.vectorDouble([0.003, 0.0075])
euribor_3m = analytics.YieldCurveBootstrapper.compute(refdate, 'EUR3M_DC', enums.DayCounter.ACT365_FIXED,  
                                                 instruments, quotes, eonia)

In [ ]:
# sample 6M EURIBOR curve with ois bootstrapping and the 3M EURIBOR curve as the basis index
quotes = analytics.vectorDouble([0.003, 0.006])
# the basis swap is used instead of the ir swap
instruments[1] = basis_swap
euribor_6m = analytics.YieldCurveBootstrapper.compute(refdate, 'EUR6M_DC', enums.DayCounter.ACT365_FIXED,  
                                                 instruments, quotes, eonia, euribor_3m)

In [ ]:
# plot discount and zero rate curves using the pyvacon.marketdata.plot.curve function based on the matplotlib.pyplot library 
fig = plt.figure(figsize = (15, 5))
# discount curves
fig.add_subplot(1,2,1) # used to get both figures in one row
mkt_plot.curve(eonia, range(1,10*365,30), refdate)
mkt_plot.curve(euribor_3m, range(1,10*365,30), refdate)
mkt_plot.curve(euribor_6m, range(1,10*365,30), refdate)

# zero rates
fig.add_subplot(1,2,2)
mkt_plot.curve(eonia, range(1,10*365,30), refdate, True)
mkt_plot.curve(euribor_3m, range(1,10*365,30), refdate, True)
mkt_plot.curve(euribor_6m, range(1,10*365,30), refdate, True)
plt.tight_layout()

### EUR market data

We now present the calibration of EUR curves based on actual market data. The input quotes as well as the instrument definition and conventions are provided in an input csv. file and are loaded into a pandas data frame object:

In [ ]:
# set holiday calendar 
holidays = analytics.SimpleHolidayCalendar('GER_HOL')
#holidays.setWeekdayAsHoliday(0) # set sunday as holiday
#holidays.setWeekdayAsHoliday(6) #set saturday as holidays

# set directory and file name for Input Quotes
dirName = "../inputdata/"
fileName = "/inputQuotes.csv"

# get instrument quotes and conventions from input .csv file 
column_names = ['Maturity','Instrument','Currency','Quote','UnderlyingIndex','UnderlyingTenor', 'UnderlyingPaymentFrequency',
                'BasisIndex','BasisTenor','BasisPaymentFrequency','PaymentFrequencyFixed','DayCountFixed',
                'DayCountFloat','DayCountBasis','RollConventionFixed','RollConventionFloat','RollConventionBasis', 'SpotLag']

dfQuotes = pandas.read_csv(dirName + fileName, sep= ";", decimal =",", skiprows=[0], header=None, names = column_names)
dfQuotes

# show data frame
#qgrid.show_grid(dfQuotes)


### EONIA curve

In [ ]:

# get input data for the EONIA curve
dfQuotesOIS = dfQuotes[dfQuotes['UnderlyingIndex'] == 'EONIA']

# set up curve parameters for bootstrapping algorithm
eoniaCurveSpec =  {'refDate': refdate_d, 
                   'curveName': 'eonia',
                   'dayCount': enums.DayCounter.ACT365_FIXED,
                   'calendar': holidays}
# get eonia curve             
eoniaCurve = bootstr.bootstrap_curve(dfQuotesOIS,eoniaCurveSpec)

### 3M EURIBOR curve

In [ ]:
# get input data for the 3M EURIBOR curve
dfQuotes3M = dfQuotes[(dfQuotes['UnderlyingIndex'] == 'EURIBOR') & (dfQuotes['UnderlyingTenor'] == '3M')]

# set up curve parameters for the 3M EURIBOR curve. The eonia curve is used for bootstrapping
euribor3MCurveSpec =  {'refDate': refdate_d, 
                      'curveName': 'euribor_3M',
                      'dayCount': enums.DayCounter.ACT365_FIXED,
                      'calendar': holidays,
                      'discountCurve': eoniaCurve}
# get 3M euribor curve              
euribor3MCurve = bootstr.bootstrap_curve(dfQuotes3M,euribor3MCurveSpec)

### 6M EURIBOR curve

In [ ]:
# get input data for the 6M EURIBOR curve
dfQuotes6M = dfQuotes[(dfQuotes['UnderlyingIndex'] == 'EURIBOR') & (dfQuotes['UnderlyingTenor'] == '6M')]

# set up curve parameters for the 6M EURIBOR curve
euribor6MCurveSpec =  {'refDate': refdate_d, 
                      'curveName': 'euribor_6M',
                      'dayCount': enums.DayCounter.ACT365_FIXED,
                      'calendar': holidays,
                      'discountCurve': eoniaCurve,
                      'basisCurve': euribor3MCurve}
# get 6M euribor curve             
euribor6MCurve = bootstr.bootstrap_curve(dfQuotes6M,euribor6MCurveSpec)

## Plotting IR curves

Here we present an alternative method for plotting curves based on the plotly library.

In [ ]:
# get output discount factors and zero rates for specified maturities
days_to_maturity = [1,12,19,26,35,68,96,127,159,187,217,249,278,370,461,551,643,735,1103,1468,1832,2196,2562,2927,3294,3659,4023,4388,5486,7312,9136,10962,14615,18268]
dates = converter.createPTimeList(refdate, days_to_maturity)
dates_d = converter.create_datetime_list(dates)

df_ois = analytics.vectorDouble()
zr_ois = analytics.vectorDouble()
eoniaCurve.value(df_ois, refdate, dates)

df_3m = analytics.vectorDouble()
zr_3m = analytics.vectorDouble()
euribor3MCurve.value(df_3m, refdate, dates)

df_6m = analytics.vectorDouble()
zr_6m = analytics.vectorDouble()
euribor6MCurve.value(df_6m, refdate, dates)

for i in range(0,len(days_to_maturity)):
    zr_ois.append(-math.log(df_ois[i])/days_to_maturity[i]*365.0)
    zr_3m.append(-math.log(df_3m[i])/days_to_maturity[i]*365.0)
    zr_6m.append(-math.log(df_6m[i])/days_to_maturity[i]*365.0)

# create data frame with curves
rates_list = {'Dates': dates_d, 
              'DiscountFactor_OIS': df_ois,
              'ZeroRate_OIS': zr_ois,
              'DiscountFactor_3M': df_3m,
              'ZeroRate_3M': zr_3m,
              'DiscountFactor_6M': df_6m,
              'ZeroRate_6M': zr_6m}

rates = pandas.DataFrame(rates_list, index = days_to_maturity)    

# plot discount factors
df_plot_data = [Scatter(name = 'Eonia', x = rates.index/365, y = rates['DiscountFactor_OIS'], mode = 'lines+markers'),
                Scatter(name = '3M Euribor', x = rates.index/365, y = rates['DiscountFactor_3M'], mode = 'lines+markers' ),
                Scatter(name = '6M Euribor', x = rates.index/365, y = rates['DiscountFactor_6M'], mode = 'lines+markers')]

df_layout = Layout(title = "Discount Factors",
               xaxis=dict(title ='Time to maturity (years)'),
               yaxis=dict(title = 'Discount Factor'))

# plot zero rates
zr_plot_data = [Scatter(name = 'Eonia', x = rates.index/365, y = rates['ZeroRate_OIS'], mode = 'lines+markers'),
                Scatter(name = '3M Euribor', x = rates.index/365, y = rates['ZeroRate_3M'], mode = 'lines+markers'),
                Scatter(name = '6M Euribor', x = rates.index/365, y = rates['ZeroRate_6M'], mode = 'lines+markers')]

zr_layout = Layout(title="Zero Rates",
               xaxis=dict(title = 'Time to maturity (years)'),
               yaxis=dict(title = 'Rate'))

iplot(Figure(data=df_plot_data, layout=df_layout))
iplot(Figure(data=zr_plot_data, layout=zr_layout))

In [ ]:
#qgrid.show_grid(rates) #uncomment to display discount factors and zero rates

## Checking bootstrapped curves

By construction, all market quotes used for bootstrapping the zero rate curves can be exactly matched. This can be used to check the correctness of the bootstrapping algorithm. Below we compare the market quotes of 3M IRS with par rates implied by the bootstrapped zero rates. 

The function getPrice calculates the present vlaue of the swap for a given swap rate. Since, by construction of the curves the original quotes are considered to be par rates, the PVs of the corresponding swaps should be zero.

In [ ]:

# get specification and quotes for test 3M IRS
dfIRS3M = dfQuotes3M[(dfQuotes3M['Instrument'] == 'IRS')]
qgrid.show_grid(dfIRS3M)
n = len(dfIRS3M.index)
prices_3mIRS = analytics.vectorDouble()
for i in range(0,n):  
    ins = bootstr.InstrumentSpec(refdate_d, dfIRS3M.iloc[i,:], holidays)
    ir_swap_pricing_data = analytics.InterestRateSwapPricingData()
    
    pay_leg_pricing_data = analytics.InterestRateSwapLegPricingData()
    pay_leg_pricing_data.discountCurve = eoniaCurve
    pay_leg_pricing_data.spec = ins.get_instrument().getPayLeg()
    pay_leg_pricing_data.fxRate = 1.0
    pay_leg_pricing_data.weight = -1.0
    
    rec_leg_pricing_data = analytics.InterestRateSwapFloatLegPricingData()
    rec_leg_pricing_data.discountCurve = eoniaCurve
    rec_leg_pricing_data.fixingCurve = euribor3MCurve
    rec_leg_pricing_data.spec = ins.get_instrument().getReceiveLeg()
    rec_leg_pricing_data.fxRate = 1.0
    rec_leg_pricing_data.weight = 1.0
    
    ir_swap_pricing_data.pricer = 'InterestRateSwapPricer'
    ir_swap_pricing_data.pricingRequest = analytics.PricingRequest()
    ir_swap_pricing_data.valDate = refdate
    ir_swap_pricing_data.setCurr('EUR')
    ir_swap_pricing_data.addLegData(pay_leg_pricing_data)
    ir_swap_pricing_data.addLegData(rec_leg_pricing_data)
    
    pr = analytics.price(ir_swap_pricing_data)
    prices_3mIRS.append(pr.getPrice())

# output 
price_data = {'Maturity': dfIRS3M['Maturity'], 
             'PV': prices_3mIRS}
price_df = pandas.DataFrame(price_data)    


# plot PV
price_plot_data = [Scatter(name = 'PV', x = price_df['Maturity'], y = price_df['PV'], mode = 'lines+markers')]

price_layout = Layout(title="Test PVs",
               xaxis=dict(title = 'Maturity'),
               yaxis=dict(title = 'PV', exponentformat = "power"))

iplot(Figure(data=price_plot_data, layout=price_layout))